In [33]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [34]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

In [35]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [36]:
BUFFER_SIZE = 1000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [37]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

print(validation_inputs.shape, validation_targets.shape)

(6000, 28, 28, 1) (6000,)


In [38]:
input_size = 784
output_size = 10

hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),

    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
NUM_EPOCHS = 30

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data,
          epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          )

Epoch 1/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.7804 - loss: 0.7449 - val_accuracy: 0.9352 - val_loss: 0.2284
Epoch 2/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9395 - loss: 0.2081 - val_accuracy: 0.9440 - val_loss: 0.1862
Epoch 3/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9535 - loss: 0.1546 - val_accuracy: 0.9517 - val_loss: 0.1568
Epoch 4/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9630 - loss: 0.1215 - val_accuracy: 0.9608 - val_loss: 0.1370
Epoch 5/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9705 - loss: 0.0999 - val_accuracy: 0.9613 - val_loss: 0.1252
Epoch 6/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9736 - loss: 0.0876 - val_accuracy: 0.9633 - val_loss: 0.1181
Epoch 7/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9777 - loss: 0.0743 - val_accuracy: 0.9683 - val_loss: 0.1104
Epoch 8/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9799 - loss: 0.0650 - val_accuracy

In [44]:
test_loss, test_accuracy = model.evaluate(test_data)

print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.9702 - loss: 0.1048
Test loss: 0.10. Test accuracy: 97.02%


In [54]:
image = cv2.imread('/content/liczba4.png', cv2.IMREAD_GRAYSCALE)

image = cv2.resize(image, (28,28))

image = 255 - image

image = image / 255.0

image = image.reshape(1, 28, 28, 1)

prediction = model.predict(image)

predicted_class = np.argmax(prediction)

print("Model rozpoznal cyfre jako: ", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Model rozpoznal cyfre jako:  5
